In this project, we want to use SQL to make a database of students' schedules and use a database of major requirements for data science majors to make a schdule builder. We want to take advantage of relational databases here. Let's start by structuring out our databases. 

In [13]:
from sqlalchemy import create_engine
import psycopg2

db_params = {
    "dbname": "postgres",
    "user": "madikloberdanz",
    "password": "3141",
    "host": "localhost",
    "port": "5433"
}

# Create a SQLAlchemy engine to connect to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Create a cursor object for executing SQL queries
c = conn.cursor()

# Courses Table
c.execute("""CREATE TABLE Courses (
    course_id INT PRIMARY KEY,
    course_name TEXT,
    prereq_id INT,
    term_offered TEXT,
    time_offered TEXT)""")

# Students Table - the earned_credits and ongoing_credits fields are arrays
c.execute("""CREATE TABLE Students (
    student_id INT PRIMARY KEY,
    student_name TEXT,
    years_left INT,
    earned_credits TEXT[],
    ongoing_credits TEXT[])""")

# Establish relations
c.execute("""ALTER TABLE Courses
ADD CONSTRAINT fk_prerequisite
FOREIGN KEY (prereq_id)
REFERENCES Courses (course_id)""")


Now that we've established our two tables and the relationships between them, let's make sure we are able to insert students, pull information from a student, update a student's information (for example, if they complete a class or change their schedule), and delete a student (if they drop the major).

In [14]:
# Add a student
def add_student(student_id, student_name, years_left, earned_credits, ongoing_credits):
    query = """INSERT INTO Students (student_id, student_name, years_left, earned_credits, ongoing_credits)
               VALUES (%s, %s, %s, %s, %s)"""
    values = (student_id, student_name, years_left, earned_credits, ongoing_credits)
    c.execute(query, values)
    conn.commit()

# Get information about a student
def get_earned_credits(student_id):
    query = """SELECT 
            ARRAY(earned_credits) 
            FROM Students WHERE student_id=%s"""
    values = (student_id,)
    c.execute(query, values)
    return c.fetchall()

# Update student info
def update_courses(student_id, course_taken):
    query = """UPDATE Students
               SET earned_credits = ARRAY_APPEND(earned_credits, %s)
               SET ongoing_credits = ARRAY_REMOVE(earned_credits, %s)
               WHERE student_id = %s"""
    values = (course_taken, student_id)
    c.execute(query, values)
    conn.commit()

# Delete a student
def delete_student(student_id):
    query = """DELETE FROM Students WHERE student_id=%s"""
    values = (student_id,)
    c.execute(query, values)
    conn.commit()



Of course, we also need to make similar functions for the classes database.

In [ ]:
# Courses Table
c.execute("""CREATE TABLE Courses (
    course_id INT PRIMARY KEY,
    course_name TEXT,
    prereq_id INT[],
    term_offered TEXT[],
    time_offered TEXT[])""")

# Add a class
def add_course(course_id, course_name, prereq_id, term_offered, time_offered):
    query = """INSERT INTO Courses (course_id, course_name, prereq_id, term_offered, time_offered))
               VALUES (%s, %s, %s, %s, %s)"""
    values = (course_id, course_name, prereq_id, term_offered, time_offered)
    c.execute(query, values)
    conn.commit()

# Get all information about a course
def get_course_info(course_id):
    query = """SELECT * 
            FROM Courses WHERE course_id=%s"""
    values = (course_id,)
    c.execute(query, values)
    return c.fetchall()

# Update student info
def update_courses(student_id, course_taken):
    query = """UPDATE Students
               SET earned_credits = ARRAY_APPEND(earned_credits, %s)
               SET ongoing_credits = ARRAY_REMOVE(earned_credits, %s)
               WHERE student_id = %s"""
    values = (course_taken, student_id)
    c.execute(query, values)
    conn.commit()

# Delete a student
def delete_student(student_id):
    query = """DELETE FROM Students WHERE student_id=%s"""
    values = (student_id,)
    c.execute(query, values)
    conn.commit()

Now, let's actually make a schedule for a student, checking for time conflicts and pre-requisites.

In [16]:
def make_schedule(student_id):
    query = ("""SELECT course_name
    FROM Courses
    WHERE prereq_id IS NULL
    OR prereq_id IN (
        SELECT course_id
        FROM Courses
        WHERE course_id IN (SELECT course_id FROM Courses WHERE prereq_id IS NULL)
        WHERE student_id=%s
    )""")
    values = (student_id,)
    c.execute(query, values)
